# Obtain NDWI using available EOplaza services

In this notebook user can find ways to execute simple process like ndwi already available in EOplaza. Service available [here](https://portal.terrascope.be/catalogue/app-details/13).

In [1]:
# import necessary packages
import openeo
from openeo.api.process import Parameter
import json
from pathlib import Path
import matplotlib.pyplot as plt
import rasterio
import numpy as np

# connect with the backend
eoconn = openeo.connect("openeo.vito.be").authenticate_oidc()

Authenticated using refresh token.


User can choose among different backend available [here](https://hub.openeo.org/) to connect to the backend. Rrgarding the authentication process OpenID connect (oidc) is recommended, but not always straightforward to use. In cases where you are unable to connect with the backend use basic authentication method explained [here](https://openeo.org/documentation/1.0/authentication.html#openid-connect).

In [2]:
# function to load geojson file
def read_json(path: Path) -> dict:
    with open(path) as input:
        field = json.load(input)
        input.close()
    return field

Since this is an already published service available service, they need not be concerned with selecting the backend. They can directly execute the process by providing time and area of interest.

In [3]:
#load datacube
date = ["2021-05-12","2021-06-12"]
aoi = read_json("./aoi/cologne_aoi.geojson")

# using an available EOplaza service
ndwi = eoconn.datacube_from_process("NDWI", namespace="vito", date=date
                                        ,polygon=aoi)

/home/pratixa/.local/lib/python3.6/site-packages/openeo/metadata.py:252: UserWarning: No cube:dimensions metadata
  complain("No cube:dimensions metadata")


Not all the available service requires updating metadata. If a service lack metadata then performing further computation on output of the service could be an issue. In such case user can update the metadata based on it's status.

In [4]:
# check available information is available in metadata or not
ndwi.metadata

In [8]:
# including temporal metadate to our dataset inorder to peform temporal reduction
from openeo.metadata import CollectionMetadata

ndwi.metadata = CollectionMetadata({"cube:dimensions":{"t":{"type":"temporal"}}})
ndwi.metadata

Once the metadata is updated you can perform further operations like subtraction or sum etc. An example of similar case is provided in a seperate notebook named: flood_ndwi.ipynb.

Since now we have details on temporal dimension we can perform dimension reduction. As we loaded our collection for specific time intervals, it can include multiple datacubes. Thus [reduce_dimension](https://processes.openeo.org/#reduce_dimension) applies a reducer to a data cube dimension by collapsing all the pixel values along the time dimension into an output value computed by the reducer.

In [9]:
# perform temporal dimension reduction
Rndwi = ndwi.reduce_dimension(dimension="t", reducer="mean")

Once the process is completed, you can also save it as your process using [save_user_defined_process](https://open-eo.github.io/openeo-python-client/udp.html) that can later be used for a similar task. Otherwise, you can download the result either by direct download (in case of the small spatial extent with few processing) or perform create a [batch job](https://open-eo.github.io/openeo-python-client/batch_jobs.html) in case it is a heavy task over a large extent.

In [10]:
#download your result
Rndwi.download("ndwi.tiff")